In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Đọc dữ liệu từ file CSV
data = pd.read_csv('OnlineRetail.csv')

In [ ]:
# Xây dựng ma trận term-document bằng TF-IDF cho mô hình Content-Based Filtering
tfidf = TfidfVectorizer(stop_words='english')
product_matrix = tfidf.fit_transform(data['Description'])

In [ ]:
# Tính toán ma trận tương đồng cosine giữa các sản phẩm
product_sim_matrix = cosine_similarity(product_matrix)

In [ ]:
# Hàm gợi ý sản phẩm dựa trên Collaborative Filtering
def collaborative_filtering_recommendation(customer_id, data):
    customer_data = data[data['CustomerID'] == customer_id]
    if customer_data.empty:
        print('Khách hàng không có dữ liệu mua hàng.')
        return
    else:
        customer_products = set(customer_data['StockCode'])
        recommended_products = set()

        for product in customer_products:
            similar_products = data[data['StockCode'] == product]['Description']
            recommended_products.update(similar_products)

        recommended_products -= customer_products
        return recommended_products

In [ ]:
# Hàm gợi ý sản phẩm dựa trên Content-Based Filtering
def content_based_recommendation(product_name, data):
    product_idx = data[data['Description'] == product_name].index[0]
    similar_products_idx = sorted(
        list(enumerate(product_sim_matrix[product_idx])),
        key=lambda x: x[1],
        reverse=True
    )
    similar_products = [data.iloc[idx]['Description'] for idx, _ in similar_products_idx[1:6]]
    return similar_products


In [ ]:
# Gợi ý sản phẩm cho khách hàng dựa trên kết hợp Collaborative Filtering và Content-Based Filtering
def hybrid_recommendation(customer_id, data, product_name):
    collaborative_products = collaborative_filtering_recommendation(customer_id, data)
    content_based_products = content_based_recommendation(product_name, data)

    recommended_products = set(collaborative_products) | set(content_based_products)
    return recommended_products

In [ ]:
# Gợi ý sản phẩm cho một khách hàng cụ thể và một sản phẩm cụ thể
customer_id = 'Mã khách hàng cần gợi ý'
product_name = 'Sản phẩm cần gợi ý'
recommended_products = hybrid_recommendation(customer_id, data, product_name)

In [ ]:
# In ra kết quả gợi ý sản phẩm
print('Các sản phẩm gợi ý:')
for product in recommended_products:
    print(product)
